In [18]:
import entropy as et
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from collections import Counter

In [3]:
data = pd.read_csv('../data/sherlock_current_wtext.csv')

In [4]:
def calculate_frequency(vector):
    vector = (vector.T/vector.sum(axis=1)).T
    return vector

In [5]:
def jsd(vector):
    return et.jensen_shannon_divergence(vector)

In [6]:
def create_df_time(df, time):
    return df[df.PublishDate.str[:7] == time]

In [7]:
data.columns.values

array(['Additional_Tags', 'Archive_Warnings', 'Author', 'Bookmarks',
       'Category', 'Chapters', 'Characters', 'Comments', 'CompleteDate',
       'Fandoms', 'Hits', 'Kudos', 'Language', 'Notes', 'PublishDate',
       'Rating', 'Relationship', 'Summary', 'Text', 'Title', 'Words'], dtype=object)

In [8]:
len(data)

204509

In [14]:
timelist = df.PublishDate.drop_duplicates().tolist()
timelist = [str(i)[:7] for i in timelist]
timelist = sorted(list(set(timelist)))[3:]

In [10]:
df_ts = create_df_time(data, "2015-11")

In [12]:
df_ts = df_ts.astype(str)
df_ts = df_ts.groupby(['Additional_Tags', 'Archive_Warnings', 'Author', 'Bookmarks',\
       'Category', 'Chapters', 'Characters', 'Comments', 'CompleteDate',\
       'Fandoms', 'Hits', 'Kudos', 'Language', 'PublishDate',\
       'Rating', 'Relationship', 'Summary', 'Title', 'Words'])['Text'].apply(','.join).reset_index()

In [61]:
text = df_ts[['Text']]
text['Text'] = text['Text'].str.replace('A', '')

/Users/jingy/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app


In [62]:
corpus = text.Text.str.lower().str.split()[1]
corpus = [re.sub(r'[^A-Za-z0-9]+', '', x) for x in corpus]

In [63]:
c = Counter(corpus)
top = [i[0] for i in c.most_common(1000)]

In [64]:
print len(text)
text = text.drop_duplicates()
print len(text)

799
656


In [67]:
vectorizer = CountVectorizer(min_df=1)

In [ ]:
for i in range(len(text) - 1):
    jsds = []
    text1 = text.iloc[i].Text.lower().split()
    text1 = [re.sub(r'[^A-Za-z0-9]+', '', x) for x in text1]
    text1 = ' '.join([word for word in text1 if word in top])

    text2 = text.iloc[i+2].Text.lower().split()
    text2 = [re.sub(r'[^A-Za-z0-9]+', '', x) for x in text2]
    text2 = ' '.join([word for word in text2 if word in top])
    
    try:
        text_vector = vectorizer.fit_transform([text1, text2]).toarray().astype(float)
        text_vector = calculate_frequency(text_vector)
    except:
        print 'invalid entry'
    jsds.append(jsd(text_vector))
    

In [ ]:
print jsds

In [335]:
# vectorizer = TfidfVectorizer(min_df=1)
# text_vector =  vectorizer.fit_transform([original1, original2]).toarray().astype(float)